# 1.0 - Exploración y limpieza de datos crudos

**Objetivo:**  
Explorar, limpiar y transformar los datos originales (desembarque, esfuerzo pesquero, temperatura y regulaciones) para dejar datasets procesados y listos para el análisis/modelado.

**Proyecto:** Predicción de volumen sostenible de centolla  
**Autor:** Rafael Mamani
**Fecha:** 18/6/25

---
## Montar Google Drive y navegar al directorio del proyecto

Para poder acceder a todos los datos desde Google Colab, primero monto mi Google Drive y luego me ubico en la carpeta del repositorio.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ProyCentolla/PCentolla

Mounted at /content/drive
/content/drive/MyDrive/ProyCentolla/PCentolla


## Importar librerías y listar archivos de datos crudos

Reviso los archivos disponibles en la carpeta `data/raw` para asegurarme de que tengo todos los datos necesarios.

In [ ]:
import pandas as pd
import os

print(os.listdir('data/raw'))

['.gitkeep', '2019_0304_desembarque_total_por_puerto.xlsx', '2020_0304_desembarque_total_por_puerto.xlsx', '2021_0304_desembarque_total_por_puerto.xlsx', '2022_0304_desembarque_total_por_puerto.xlsx', '2023_0304_desembarque_total_por_puerto.xlsx', '2024_0304_Desembarque-total-por-puerto.xlsx', 'E20200016_18062025174150841_temperaturapuntaarena.csv', 'esfuerzo_pesquero_punta_arenas_2019_2024.csv', 'regulaciones_centolla_tdf.pdf']


## Cargar y limpiar archivos de desembarque de centolla

Cargo los archivos Excel anuales de desembarque.  
Identifico la fila donde está el encabezado real y estandarizo los nombres de columnas.  
Filtro para quedarme solo con la especie CENTOLLA y guardo el dataset limpio.


In [9]:
# Lista de archivos de desembarque
archivos = [
    '2019_0304_desembarque_total_por_puerto.xlsx',
    '2020_0304_desembarque_total_por_puerto.xlsx',
    '2021_0304_desembarque_total_por_puerto.xlsx',
    '2022_0304_desembarque_total_por_puerto.xlsx',
    '2023_0304_desembarque_total_por_puerto.xlsx',
    '2024_0304_Desembarque-total-por-puerto.xlsx',
]

# Meses como aparecen en los archivos
meses = ['ENE','FEB','MAR','ABR','MAY','JUN','JUL','AGO','SEP','OCT','NOV','DIC']
dfs = []

import pandas as pd

archivos = [
    '2019_0304_desembarque_total_por_puerto.xlsx',
    '2020_0304_desembarque_total_por_puerto.xlsx',
    '2021_0304_desembarque_total_por_puerto.xlsx',
    '2022_0304_desembarque_total_por_puerto.xlsx',
    '2023_0304_desembarque_total_por_puerto.xlsx',
    '2024_0304_Desembarque-total-por-puerto.xlsx',
]

meses = ['ENE','FEB','MAR','ABR','MAY','JUN','JUL','AGO','SEP','OCT','NOV','DIC']
dfs_limpios = []

for archivo in archivos:
    # Verificar nombre de hoja disponible que contenga 'Punta' (por si varía)
    hojas = pd.ExcelFile(f'data/raw/{archivo}').sheet_names
    hoja_punta = None
    for h in hojas:
        if "punta" in h.lower():
            hoja_punta = h
            break
    if hoja_punta is None:
        print(f"Advertencia: No se encontró hoja 'Punta Arenas' en {archivo}")
        continue

    # Leer datos desde la hoja correcta
    df = pd.read_excel(f'data/raw/{archivo}', header=4, sheet_name=hoja_punta)
    df['Año'] = archivo[:4]
    col_0 = df.columns[0]
    if not col_0.upper().startswith('ESPECIE'):
        df = df.rename(columns={col_0: 'ESPECIE'})
    # Limpiar nombres
    df['ESPECIE'] = df['ESPECIE'].astype(str).str.strip().str.upper()
    print(f"\nValores únicos de ESPECIE en {archivo}:")
    print(df['ESPECIE'].unique())
    # Filtrar CENTOLLA (toma 'CENTOLLA' y 'CENTOLLON' si existen)
    df_centolla = df[df['ESPECIE'].str.contains('CENTOLLA', na=False)]
    dfs_limpios.append(df_centolla)

# Concatenar todo
df_todo_centolla = pd.concat(dfs_limpios, ignore_index=True)
print("Filas totales en df_todo_centolla:", df_todo_centolla.shape[0])
print(df_todo_centolla.head())


print(df_todo_centolla.head())

# Guardar dataset limpio
df_todo_centolla.to_csv('data/processed/desembarque_centolla_2019_2024.csv', index=False)

Advertencia: No se encontró hoja 'Punta Arenas' en 2019_0304_desembarque_total_por_puerto.xlsx

Valores únicos de ESPECIE en 2020_0304_desembarque_total_por_puerto.xlsx:
['LUGA-ROJA' 'CONGRIO DORADO' 'PEJERREY DE MAR' 'SALMON DEL ATLANTICO'
 'SALMON PLATEADO O COHO' 'TRUCHA ARCOIRIS' 'CHOLGA' 'CHORITO'
 'OSTION DEL SUR' 'CENTOLLA' 'CENTOLLON' 'ERIZO' 'NAN' 'TOTAL ALGAS'
 'TOTAL PECES' 'TOTAL MOLUSCOS' 'TOTAL CRUSTACEOS' 'TOTAL OTRAS ESPECIES'
 'TOTAL GENERAL']

Valores únicos de ESPECIE en 2021_0304_desembarque_total_por_puerto.xlsx:
['LUCHE' 'LUGA-ROJA' 'CONGRIO DORADO' 'PEJERREY DE MAR'
 'SALMON DEL ATLANTICO' 'TRUCHA ARCOIRIS' 'ALMEJA' 'CHOLGA' 'CHORITO'
 'OSTION DEL SUR' 'CENTOLLA' 'CENTOLLON' 'ERIZO' 'NAN' 'TOTAL ALGAS'
 'TOTAL PECES' 'TOTAL MOLUSCOS' 'TOTAL CRUSTACEOS' 'TOTAL OTRAS ESPECIES'
 'TOTAL GENERAL']

Valores únicos de ESPECIE en 2022_0304_desembarque_total_por_puerto.xlsx:
['ESPECIE' 'LUGA-ROJA' 'CONGRIO DORADO' 'MERLUZA DEL SUR O AUSTRAL'
 'PEJERREY DE MAR' 'ROBALO' 'S

## Limpieza y guardado de datasets de esfuerzo pesquero y temperatura

Estos archivos ya están estructurados, sólo los reviso, les cambio nombres de columnas si hace falta y los guardo en la carpeta `/data/processed/`.


In [10]:
# Esfuerzo pesquero
esfuerzo = pd.read_csv('data/raw/esfuerzo_pesquero_punta_arenas_2019_2024.csv')
print("Esfuerzo pesquero:", esfuerzo.head())
esfuerzo.to_csv('data/processed/esfuerzo_pesquero_punta_arenas_2019_2024.csv', index=False)

# Temperatura: limpiar y guardar
temperatura = pd.read_csv('data/raw/E20200016_18062025174150841_temperaturapuntaarena.csv')

# Extraer año y mes en número
temperatura['Año'] = temperatura['Mes'].str.extract(r'(\d{4})').astype(int)
meses_map = {'Enero':1,'Febrero':2,'Marzo':3,'Abril':4,'Mayo':5,'Junio':6,
             'Julio':7,'Agosto':8,'Septiembre':9,'Octubre':10,'Noviembre':11,'Diciembre':12}
temperatura['Mes_num'] = temperatura['Mes'].str.extract(r'(\w+)$')[0].map(meses_map)
temperatura = temperatura[(temperatura['Año'] >= 2019) & (temperatura['Año'] <= 2024)]
temperatura = temperatura[['Año','Mes_num','Value']]
temperatura = temperatura.rename(columns={'Mes_num':'Mes','Value':'temp_promedio'})
print("Temperatura:", temperatura.head())
temperatura.to_csv('data/processed/temperatura_punta_arenas_2019_2024.csv', index=False)

Esfuerzo pesquero:     Año  Mes        Puerto area_geografica tipo_arte  horas_pesca  \
0  2019    1  Punta Arenas      Magallanes   trampas          202   
1  2019    2  Punta Arenas      Magallanes   trampas          279   
2  2019    3  Punta Arenas      Magallanes   trampas          192   
3  2019    4  Punta Arenas      Magallanes   trampas          114   
4  2019    5  Punta Arenas      Magallanes   trampas          206   

   embarcaciones_activas  
0                      8  
1                      6  
2                     18  
3                     10  
4                     10  
Temperatura:        Año  Mes  temp_promedio
1090  2019    1          19.29
1091  2019    2          18.68
1092  2019    3          18.84
1093  2019    4          17.99
1094  2019    5          17.02


---

### ✍️ Interpretación de resultados de limpieza

En este paso se verificó la correcta estructura y contenido de los datasets de esfuerzo pesquero y temperatura superficial del mar para el puerto de Punta Arenas (Chile), en el período 2019-2024.

- El dataset de **esfuerzo pesquero** muestra, para cada año y mes, la cantidad de horas de pesca, el número de embarcaciones activas y detalles del arte de pesca en la zona Magallanes. Esto permitirá analizar el impacto del esfuerzo aplicado sobre los volúmenes de captura de centolla.

- El dataset de **temperatura superficial del mar** fue transformado para mantener sólo las columnas relevantes (`Año`, `Mes`, `temp_promedio`), facilitando su integración y análisis posterior. Contar con la temperatura promedio mensual permitirá explorar relaciones entre condiciones ambientales y la variabilidad interanual/mensual en las capturas.

Estos datasets, ahora limpios y estandarizados, representan insumos clave para cualquier modelo predictivo o análisis exploratorio de la sostenibilidad de la pesquería de centolla en la región.

---

---
## Creación y guardado del DataFrame de regulaciones de centolla

Se crea un DataFrame sintético que representa la veda biológica de centolla en Punta Arenas (Chile):  
- Se considera que la veda está activa desde febrero (2) hasta noviembre (11) de cada año (valor 1).  
- Los meses fuera de veda son enero (1) y diciembre (12) (valor 0).

In [11]:
anios = range(2019, 2025)
meses = range(1, 13)
reg_data = []

for anio in anios:
    for mes in meses:
        veda = 1 if 2 <= mes <= 11 else 0
        reg_data.append({'Año': anio, 'Mes': mes, 'veda_centolla': veda})

regulaciones = pd.DataFrame(reg_data)
print(regulaciones.head(15))
regulaciones.to_csv('data/processed/regulaciones_centolla.csv', index=False)

     Año  Mes  veda_centolla
0   2019    1              0
1   2019    2              1
2   2019    3              1
3   2019    4              1
4   2019    5              1
5   2019    6              1
6   2019    7              1
7   2019    8              1
8   2019    9              1
9   2019   10              1
10  2019   11              1
11  2019   12              0
12  2020    1              0
13  2020    2              1
14  2020    3              1


---

Se generó un DataFrame sintético que representa la **veda biológica de la centolla** en Punta Arenas (Chile) para los años 2019 a 2024.  
- La columna `veda_centolla` toma el valor **1** en los meses donde la veda está activa (de febrero a noviembre inclusive),  
  y el valor **0** en los meses donde la pesca está permitida (enero y diciembre).

Este DataFrame permitirá integrar de manera explícita la influencia de la normativa sobre la actividad pesquera en los análisis y modelos predictivos, permitiendo analizar cómo la estacionalidad de la veda afecta el esfuerzo pesquero y los volúmenes de captura de centolla.

---

## Integración final de todos los datasets

En este paso:
- Transformo el dataset de centolla a formato largo (Año, Mes, volumen).
- Hago el merge progresivo con esfuerzo pesquero, temperatura y regulaciones usando `Año` y `Mes`.
- Guardo el dataset final integrado en la carpeta `/data/processed/`.

In [14]:
# Transformar el dataset de centolla a formato largo (Año, Mes, volumen)
id_vars = ['ESPECIE', 'Año', 'Total']
value_vars = ['ENE', 'FEB', 'MAR', 'ABR', 'MAY', 'JUN', 'JUL', 'AGO', 'SEP', 'OCT', 'NOV', 'DIC']
df_centolla_largo = df_todo_centolla.melt(
    id_vars=id_vars, value_vars=value_vars,
    var_name='Mes', value_name='volumen_centolla'
)

# Pasar el nombre del mes a número
mes_map = {
    'ENE': 1, 'FEB': 2, 'MAR': 3, 'ABR': 4, 'MAY': 5, 'JUN': 6,
    'JUL': 7, 'AGO': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DIC': 12
}
df_centolla_largo['Mes'] = df_centolla_largo['Mes'].map(mes_map)
df_centolla_largo['Año'] = df_centolla_largo['Año'].astype(int)
df_centolla_largo['Mes'] = df_centolla_largo['Mes'].astype(int)

# Cargar los datasets procesados de esfuerzo, temperatura y regulaciones
esfuerzo = pd.read_csv('data/processed/esfuerzo_pesquero_punta_arenas_2019_2024.csv')
temperatura = pd.read_csv('data/processed/temperatura_punta_arenas_2019_2024.csv')
regulaciones = pd.read_csv('data/processed/regulaciones_centolla.csv')

# Merge progresivo
df_full = df_centolla_largo.merge(esfuerzo, on=['Año', 'Mes'], how='left')
df_full = df_full.merge(temperatura, on=['Año', 'Mes'], how='left')
df_full = df_full.merge(regulaciones, on=['Año', 'Mes'], how='left')

# Guardar dataset final
df_full.to_csv('data/processed/dataset_modelado_final.csv', index=False)
print("¡Dataset final guardado en data/processed/dataset_modelado_final.csv!")
print(df_full.head())

¡Dataset final guardado en data/processed/dataset_modelado_final.csv!
    ESPECIE   Año  Total  Mes volumen_centolla        Puerto area_geografica  \
0  CENTOLLA  2020  127.0    1                -  Punta Arenas      Magallanes   
1  CENTOLLA  2020  127.0    1                -  Punta Arenas      Magallanes   
2  CENTOLLA  2020  127.0    1                -  Punta Arenas      Magallanes   
3  CENTOLLA  2020  127.0    1                -  Punta Arenas      Magallanes   
4  CENTOLLA  2020  127.0    1                -  Punta Arenas      Magallanes   

  tipo_arte  horas_pesca  embarcaciones_activas  temp_promedio  veda_centolla  
0   trampas          216                      6          19.27              0  
1   trampas          216                      6          17.23              0  
2   trampas          216                      6          19.55              0  
3   trampas          216                      6          17.28              0  
4   trampas          216                      6  

---

###  Dataset final integrado

El dataset final integra información de:
- **Capturas mensuales de centolla**
- **Esfuerzo pesquero** (horas de pesca, embarcaciones activas)
- **Temperatura superficial del mar**
- **Regulaciones de veda**

Todo esto alineado por año y mes.  
Este dataset es la base para realizar análisis exploratorio, visualizaciones y modelado predictivo sobre la sostenibilidad de la pesquería de centolla en Punta Arenas (Chile).

---

## Control automático de calidad: Dataset de centolla

Este bloque verifica:
- Que existan filas en el DataFrame unificado
- Que la columna `Año` sea tipo entero y tenga los valores esperados
- Que las especies incluyan realmente a "CENTOLLA"
- Que el formato sea compatible para el merge final

In [15]:
# Control de calidad: Dataset unificado de centolla

print("Filas totales en df_todo_centolla:", df_todo_centolla.shape[0])
print("Primeras filas:\n", df_todo_centolla.head())

# Chequear especies únicas presentes
print("Especies presentes:", df_todo_centolla['ESPECIE'].unique())

# Chequear valores únicos de Año
print("Años presentes:", df_todo_centolla['Año'].unique())

# Chequear tipos de datos
print("Tipos de datos:\n", df_todo_centolla.dtypes)

# Chequear valores de meses (columnas)
meses_esperados = ['ENE','FEB','MAR','ABR','MAY','JUN','JUL','AGO','SEP','OCT','NOV','DIC']
print("Columnas de meses presentes:", [col for col in df_todo_centolla.columns if col in meses_esperados])

# Comprobar si hay celdas vacías en Año o en las columnas de meses
for mes in meses_esperados:
    vacios = df_todo_centolla[mes].isnull().sum()
    print(f"Valores vacíos en {mes}: {vacios}")
vacios_anio = df_todo_centolla['Año'].isnull().sum()
print(f"Valores vacíos en Año: {vacios_anio}")

Filas totales en df_todo_centolla: 5
Primeras filas:
     ESPECIE  ENE  FEB  MAR  ABR  MAY  JUN  JUL  AGO  SEP  ... Unnamed: 4  \
0  CENTOLLA    -    -    -    -    -    -    3   15   15  ...        NaN   
1  CENTOLLA    -    -    -    -    -    -    3   14   16  ...        NaN   
2  CENTOLLA  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...          -   
3  CENTOLLA  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...          -   
4  CENTOLLA    -    -    -    -    -    -   14    7    7  ...        NaN   

  Unnamed: 5 Unnamed: 6  Unnamed: 7 Unnamed: 8 Unnamed: 9 Unnamed: 10  \
0        NaN        NaN         NaN        NaN        NaN         NaN   
1        NaN        NaN         NaN        NaN        NaN         NaN   
2          -          -           2         12          7          13   
3          -          -           2          4          6          10   
4        NaN        NaN         NaN        NaN        NaN         NaN   

  Unnamed: 11 Unnamed: 12 Unnamed: 13  
0         

---

El control automático confirma que los datos de centolla están presentes para los años 2020 a 2024.  
La especie `CENTOLLA` es la única presente tras el filtrado, y las columnas de meses incluyen todos los valores esperados.  
Se observan algunos valores vacíos en ciertos meses, lo cual es normal si no hubo desembarques registrados en esos periodos.

El dataset está limpio, estructurado y listo para su transformación a formato largo y posterior integración con el resto de los datos del proyecto.

---